In [1]:
import pandas as pd
import os
import datetime
from glob import glob

In [126]:
PATH = '.'
EXT = '*.csv'
all_BIB_files = [file
                 for path, subdir, files in os.walk(PATH)
                 for file in glob(os.path.join(path, EXT))]
BA_files = 'SS'
all_BA_files = [y for y in all_BIB_files if BA_files in y]
print(all_BA_files)

['.\\SS\\PASt Stephens roof.csv', '.\\SS\\St StephensIndoorHall.csv', '.\\SS\\StStephensBIBoutdoor.csv', '.\\SS\\StStephensIndoorclass.csv', '.\\SS\\TtestTrial.csv']


In [140]:
df= pd.read_csv('SS\\TtestTrial.csv', parse_dates=['Date'], index_col='Date', dayfirst=True)

In [141]:
df

,"PM2.5, ug/m3","PM10, ug/m3"
Date,,
2023-01-27,1,3
2023-01-28,1,3
2023-01-29,1,3
2023-01-30,1,4
2023-01-31,1,3
2023-02-01,1,5
2023-02-02,1,3
2023-02-03,1,5
2023-02-04,2,6


In [129]:
x= df['PM2.5, ug/m3']
a=x[0:5]
a

Date
2023-01-27    1
2023-01-28    1
2023-01-29    1
2023-01-30    1
2023-01-31    1
Name: PM2.5, ug/m3, dtype: int64

In [130]:
b=x[5:10]
b

Date
2023-02-01    1
2023-02-02    1
2023-02-03    1
2023-02-04    2
2023-02-05    2
Name: PM2.5, ug/m3, dtype: int64

In [131]:
from scipy.stats import ttest_ind
from scipy import stats
res = stats.ttest_ind(a, b)
print(res)

Ttest_indResult(statistic=-1.6329931618554518, pvalue=0.14111328125000006)


In [132]:
tes= stats.ttest_rel(a, b)
print(tes)

Ttest_relResult(statistic=-1.632993161855452, pvalue=0.17780780835622126)


In [122]:
from scipy.stats import ttest_ind
def ttest(a, b):
    tscore, pvalue = ttest_ind(a, b)
    return pvalue
rolling_ttest = df \
    .filter(['PM2.5, ug/m3']) \
    .rolling(window=10).agg(ttest)\
    .rename(lambda x: f'{x}_t_test') 


df2test = pd.concat([df, rolling_ttest], axis=1)
df2test

DataError: No numeric types to aggregate

In [139]:
#For a one sample t-test it seemed more straightforward,
#as I am comparing each row to the population mean
df['PM2.5, ug/m3'].mean()

6.45

In [145]:
from scipy.stats import ttest_1samp
mean = 6.45
sig_level = 0.05

def run_test(x, mean):
    tscore, pvalue = ttest_1samp(x, mean)
    return pvalue

rolling_ttest = df \
    .filter(['PM2.5, ug/m3'])\
    .rolling(window=5).agg(run_test, mean = mean)\
    .rename(lambda x: f'{x}_t_test') 


dfttest = pd.concat([df, rolling_ttest], axis=1)
dfttest

,"PM2.5, ug/m3","PM10, ug/m3","PM2.5, ug/m3"
Date,,,
2023-01-27 00:00:00,1.0,3.0,NaN
2023-01-28 00:00:00,1.0,3.0,NaN
2023-01-29 00:00:00,1.0,3.0,NaN
2023-01-30 00:00:00,1.0,4.0,NaN
2023-01-31 00:00:00,1.0,3.0,NaN
2023-02-01 00:00:00,1.0,5.0,NaN
2023-02-02 00:00:00,1.0,3.0,NaN
2023-02-03 00:00:00,1.0,5.0,NaN
2023-02-04 00:00:00,2.0,6.0,NaN
